In [1]:
import plotly.figure_factory as ff
from typing import List, Dict


In [2]:
# -- Local imports
from utils.elasticsearch_utils import get_all_hits, flatten_dict, flatten_dict_keys, \
    get_hits_dsl_query, \
    get_hits_dict_query, update_fields_select_df, get_server_reports

In [ ]:
# -- Useful paths
OUT_DIR = "results/reports"
eDATA_FILTER_FILE = "configs/edata_fields.csv"
rDATA_FILTER_FILE = "configs/reporting_fields.csv"

In [ ]:
# -- Example of elasticsearch data fetch

# -- Simple Gather all data from elasticsearch
full_edata: List[Dict] = get_all_hits()

edata: List[Dict] = get_hits_dsl_query({
    "match": {
        "args.title": {
            "query": "andrei",
            "type": "phrase"
        }
    }
})

edata: List[Dict] = get_hits_dict_query({
    "args.title": ["andrei"], 
    "args.tasks.datasets": ["mnist", "cifar10"]
})

In [ ]:
# -- Simple fetch of report from server 
rdata: List[Dict] = get_server_reports(e_ids=["E1sZ62MBm5wd3rDHL-EF"])

In [4]:
# -- Key filters for reports or eData
rdata_keys = [] # Empty if you do not want to update keys
edata_keys = []

# eDATA columns read 
edata_keys = sorted(list(flatten_dict_keys(edata[0])))  # Use tu update / initialize

# Reporting columns read 
rdata_keys = sorted(list(flatten_dict_keys(rdata[0])))  # Use tu update / initialize

# -- Read /update them to file
# Read key filters and smarg_grop prop for eData 
e_select_df, e_select_k, e_select_sg = update_fields_select_df(None, edata_keys, 
                                                               update_file=eDATA_FILTER_FILE)

# Read key filters and smarg_grop prop for raw reporting data 
r_select_df, r_select_k, r_select_sg = update_fields_select_df(None, rdata_keys, 
                                                               update_file=rDATA_FILTER_FILE)

In [ ]:

edata: List[Dict] = get_hits_dict_query({
    "args.title": ["andrei"]}, include_keys=e_select_k)

In [ ]:
# Example of Gathering raw data from server

import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

full_report, df_return = get_server_reports(experiments=["3d_datasets_ewc_256_mlp"],
                                            include_keys=r_select_k, smart_group=r_select_sg,
                                            df_format=True)
df, report_info = df_return

optim_name = "_args.train._optimizer.__name."
optimizers = df[optim_name].unique()
seen_idxs = df["seen"].unique()

for task_idx in df["task_idx"].unique():
    df_task = df[df["task_idx"] == task_idx]

cnt = df_task.groupby(["seen", optim_name]).count()["task_idx"].describe()
select = int(cnt["75%"])

df_task["c_reporting_idx"] = df_task.groupby(["seen", optim_name]).cumcount()

ax = sns.tsplot(data=df_task, time="seen", value="_eval_trace.0.acc.2", 
                condition="_args.train._optimizer.__name.")
plt.show()


In [4]:
df.to_csv("test2.csv")